In [1]:
from utilities import *

In [2]:
from libraries import *

In [ ]:
# torch_geometric

In [3]:
import torch
from torch_geometric.data import Data


In [4]:
Pickle_path = '../Pickle_Files/'
csv_path = '../Data/'

In [5]:
User_level_feat_df = pd.read_csv(csv_path+"User_profile.csv")
sample_data = pickle_loader(Pickle_path+"URL_base_data_train_test_POC.pkl")
Follower_cnt_dict = pickle_loader(Pickle_path+"Follower_cnt_dict.pkl")
similarity_dict = pickle_loader(Pickle_path+"node_similarity_dict.pkl")

In [6]:

# assume your DataFrame is named df
tw_usr_screen_dict = User_level_feat_df.reset_index().set_index('tw_usr_screen')['index'].to_dict()


In [7]:
Follower_cnt_dict_idx = {}

for key, val in Follower_cnt_dict.items():
    Follower_cnt_dict_idx[tw_usr_screen_dict[key]] = val

In [8]:
similarity_dict_idx = {}

for user,adj in similarity_dict.items():
    inner_dict = {}
    for key,val in adj.items():
        inner_dict[tw_usr_screen_dict[key]] = val
    similarity_dict_idx[tw_usr_screen_dict[user]] = inner_dict

In [9]:
import gc

In [10]:
gc.collect()

0

In [11]:

# Define the number of nodes (users)
num_nodes = len(similarity_dict_idx)


In [12]:

# Create the edge list and edge weights
edge_list = []
# edge_weights = []
for node1, neighbors in similarity_dict_idx.items():
    for node2, weight in neighbors.items():
        if node1!=node2:
            edge_list.append((node1, node2,weight))
#         edge_weights.append(weight)



In [13]:

edge_list_tmp = sorted(edge_list, key=lambda x: x[0])


In [14]:
loops = []
for pair in edge_list_tmp:
    if pair[0]==pair[1]:
        loops.append(pair)

In [15]:
edge_weights = []
for pairs in edge_list:
    edge_weights.append(pairs[2])

In [16]:
edge_list = [w[:2] for w in edge_list_tmp]

In [17]:
sorted_dict = {k: v for k, v in sorted(Follower_cnt_dict_idx.items())}



In [18]:
features = torch.zeros((num_nodes, 2))


num_node_features = 2

for node, feature_vector in sorted_dict.items():
    try:
        features[node] = torch.tensor([int(feature_vector),int(feature_vector)])
    except:
        print(node)
        break


In [19]:


Base_graph = Data(x=features, edge_index=torch.tensor(edge_list).t().contiguous(), edge_attr=torch.tensor(edge_weights), y = torch.tensor(10000))


In [20]:
data = Base_graph  # Get the first graph object.

print()
print(data)
print('=============================================================')

# Gather some statistics about the first graph.
print(f'Number of nodes: {data.num_nodes}')
print(f'Number of edges: {data.num_edges}')
print(f'Average node degree: {data.num_edges / data.num_nodes:.2f}')
print(f'Has isolated nodes: {data.has_isolated_nodes()}')
print(f'Has self-loops: {data.has_self_loops()}')
print(f'Is undirected: {data.is_undirected()}')


Data(x=[98896, 2], edge_index=[2, 869989], edge_attr=[869989], y=10000)
Number of nodes: 98896
Number of edges: 869989
Average node degree: 8.80
Has isolated nodes: False
Has self-loops: False
Is undirected: False


### Load URL data to generate train,test datasets

In [21]:

sample_data.reset_index(drop=True,inplace=True)

sample_data['usr_screen_60'] = sample_data.apply(lambda row: tuple(set((row['rtw_usr_screen_list_60']+ row['tw_usr_screen_list_60']))), axis=1)


sample_data['usr_screen_all'] = sample_data.apply(lambda row: tuple(set((row['rtw_usr_screen_list_all']+ row['tw_usr_screen_list_all']))), axis=1)


def extract_user_ids(screen_names, mapping_dict):
    user_ids = []
    for screen_name in screen_names:
        if screen_name in mapping_dict:
            user_ids.append(mapping_dict[screen_name])
    return tuple(user_ids)


sample_data['train_user_ids'] = sample_data['usr_screen_60'].apply(lambda x: extract_user_ids( x, tw_usr_screen_dict))
sample_data['test_user_ids'] = sample_data['usr_screen_all'].apply(lambda x: extract_user_ids(x, tw_usr_screen_dict))


In [22]:


def node_feature_creation(sorted_dict,Is_url_Tweet_lst):
    features = torch.zeros((num_nodes, 2))
#     print(Is_url_Tweet_lst)
#     print(type(Is_url_Tweet_lst))
    num_node_features = 2
    cnt = 0
    for node, feature_vector in sorted_dict.items():
        try:
            if node in Is_url_Tweet_lst:
                features[node] = torch.tensor([int(feature_vector),1])
                cnt+=1
            else:
                features[node] = torch.tensor([int(feature_vector),0])
        except:
            print(node)
            break
#     print(cnt)
    return features


### Train test Graph Generation Module

SO we have static edge list and edge weight list.

we create dynamic node features
    1. Follower COunt
    2. Binary flag whether a node tweeted a URL or not

Target Y is : Retweet Count

It will vary from train to test as we take only 60% data per URl in training and remaining in test 

In [23]:
def Generate_Graph(train_nodes,y_val):


    Level_1_Neighbours = []

    Level_2_Neighbours = []

    Node_Feature_list = set()

    Node_set = set()


    for root_node in train_nodes:
        if root_node in similarity_dict_idx.keys():
            Node_set.add(root_node)

            follower_cnt = int(Follower_cnt_dict_idx.get(root_node,0))

            Node_Feature_list.add((root_node,follower_cnt,1))   ### add 2 features - 1 is follower cnt, 2 is binary flag that they tweeted url

            neighbours = similarity_dict_idx.get(root_node)
            for neighbour, weight in neighbours.items():
                if neighbour!=root_node:
                    Node_set.add(neighbour)
                    Level_1_Neighbours.append((root_node,neighbour,weight))
                    follower_cnt = int(Follower_cnt_dict_idx.get(neighbour,0))
                    if neighbour not in train_nodes:
                        Node_Feature_list.add((neighbour,follower_cnt,0))   ### add 2 features - 1 is follower cnt, 2 is binary flag that they tweeted url

                else:
                    continue
                if neighbour in  similarity_dict_idx.keys():
                    neighbour_l2 = similarity_dict_idx.get(neighbour)
                    for neighbour2, weight2 in neighbour_l2.items():
                        if neighbour!=neighbour2:
                            Node_set.add(neighbour2)
                            Level_2_Neighbours.append((neighbour,neighbour2,weight2))
                            follower_cnt = int(Follower_cnt_dict_idx.get(neighbour2,0))
                            if neighbour2 not in train_nodes:
                                Node_Feature_list.add((neighbour2,follower_cnt,0))   ### add 2 features - 1 is follower cnt, 2 is binary flag that they tweeted url

                        else:
                            continue
                
                
    #### IN aboe code we first identify immediate neighbours of nodes who tweeted a URL, Level_1_Neighbours
    #### then we first second level enighbours - Friend of Friend - stored in  Level_2_Neighbours          
    
    edge_list_1 = Level_1_Neighbours+Level_2_Neighbours  
    node_index_idx = {}

    for index,node in enumerate(Node_set):
    #     print(index,node)
        node_index_idx[node] = index

    edge_list_dedup_1 = list(set(edge_list_1))

    edge_list_dedup_2 = [(node_index_idx[w[0]],node_index_idx[w[1]],w[2]) for w in edge_list_dedup_1]
    edge_list_dedup = sorted(edge_list_dedup_2, key=lambda x: x[0])
    edge_weights = []
    for pairs in edge_list_dedup:
        edge_weights.append(pairs[2])
    edge_list = [w[:2] for w in edge_list_dedup]
    Node_Feature_list_tmp = []

    for node in Node_Feature_list:
        new_key = node_index_idx[node[0]]
        Node_Feature_list_tmp.append((new_key,node[1],node[2]))
    Node_Feature_list_sorted = sorted(Node_Feature_list_tmp, key=lambda x: x[0])
    features = torch.zeros((len(Node_Feature_list_sorted), 2))


    num_node_features = 2

    for node, feature_vector in enumerate(Node_Feature_list_sorted):
        try:
            features[node] = torch.tensor([int(feature_vector[1]),int(feature_vector[2])])
        except:
            print(node)
            break
    Num_nodes = len(features)
    graph_trn = Data(x=features, edge_index=torch.tensor(edge_list).T, edge_attr=torch.tensor(edge_weights), y = torch.tensor(y_val))
    return graph_trn


In [25]:
import numpy as np



In [28]:
import random

def exract_rand_60pc(my_list):
    my_list = list(my_list)

#     my_list = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

    n = len(my_list)
    k = int(0.6 * n)  # select 60% of elements

    random.shuffle(my_list)  # shuffle the list randomly

    return my_list[:k] 

In [29]:
Dataset_train = []
Dataset_test = []

for index, row in sample_data.iterrows():
#     uniform_var = np.random.rand() 
#     if uniform_var>0.75:
        feat_list_train  = exract_rand_60pc(row["train_user_ids"])
        y_val_train = int(row['retweet_cnt_sum_60'])
        y_val_test = int(row['retweet_cnt_sum_all'])
        
#         graph_trn = Data(x=feat_list_train, edge_index=torch.tensor(edge_list).T, edge_attr=torch.tensor(edge_weights), y =y_val_train )
        graph_trn = Generate_Graph(feat_list_train,np.log10(1+y_val_train))
        Dataset_train.append(graph_trn)
#     uniform_var = np.random.rand() 

#     if uniform_var>0.75:
        feat_list_test  = exract_rand_60pc(row["test_user_ids"])
        y_val_test = int(row['retweet_cnt_sum_all'])   ### lets try - np.log(10)
#         graph_test = Data(x=feat_list_test, edge_index=torch.tensor(edge_list).T, edge_attr=torch.tensor(edge_weights), y =y_val_test )
        graph_test = Generate_Graph(feat_list_test,np.log10(1+y_val_test))

        Dataset_test.append(graph_test)
#         break
    
    
    
    
    

In [30]:
import torch
from torch_geometric.data import Data


In [31]:
import torch_geometric

In [32]:
from torch_geometric.loader import DataLoader

train_loader = DataLoader(Dataset_train, batch_size=16, shuffle=True)
test_loader = DataLoader(Dataset_test, batch_size=16, shuffle=False)

for step, data in enumerate(train_loader):
    print(f'Step {step + 1}:')
    print('=======')
    print(f'Number of graphs in the current batch: {data.num_graphs}')
    print(data)
    print()

Step 1:
Number of graphs in the current batch: 16
DataBatch(x=[4205, 2], edge_index=[2, 7896], edge_attr=[7896], y=[16], batch=[4205], ptr=[17])

Step 2:
Number of graphs in the current batch: 16
DataBatch(x=[5022, 2], edge_index=[2, 9846], edge_attr=[9846], y=[16], batch=[5022], ptr=[17])

Step 3:
Number of graphs in the current batch: 16
DataBatch(x=[4537, 2], edge_index=[2, 8607], edge_attr=[8607], y=[16], batch=[4537], ptr=[17])

Step 4:
Number of graphs in the current batch: 16
DataBatch(x=[3828, 2], edge_index=[2, 8442], edge_attr=[8442], y=[16], batch=[3828], ptr=[17])

Step 5:
Number of graphs in the current batch: 16
DataBatch(x=[5037, 2], edge_index=[2, 9532], edge_attr=[9532], y=[16], batch=[5037], ptr=[17])

Step 6:
Number of graphs in the current batch: 16
DataBatch(x=[4128, 2], edge_index=[2, 8602], edge_attr=[8602], y=[16], batch=[4128], ptr=[17])

Step 7:
Number of graphs in the current batch: 16
DataBatch(x=[3590, 2], edge_index=[2, 6830], edge_attr=[6830], y=[16], bat

In [33]:
from torch.nn import Linear
import torch.nn.functional as F
from torch_geometric.nn import GCNConv,GraphConv
from torch_geometric.nn import global_mean_pool,global_max_pool


class GCN(torch.nn.Module):
    def __init__(self, hidden_channels):
        super(GCN, self).__init__()
        torch.manual_seed(23311)
        self.conv1 = GCNConv(num_node_features, hidden_channels)
        self.conv2 = GraphConv(hidden_channels, hidden_channels)
        self.conv3 = GraphConv(hidden_channels, hidden_channels)
        self.conv4 = GraphConv(hidden_channels, hidden_channels)
        self.conv5 = GraphConv(hidden_channels, hidden_channels)
        
        self.lin = Linear(hidden_channels, 1)

    def forward(self, x, edge_index, batch):
        # 1. Obtain node embeddings 
        x = self.conv1(x, edge_index)
        x = x.relu()
        x = self.conv2(x, edge_index)
        x = x.relu()
        x = self.conv3(x, edge_index)
        x = x.relu()
        x = self.conv4(x, edge_index)
        x = x.relu()
        x = self.conv5(x, edge_index)

        # 2. Readout layer
        x = global_mean_pool(x, batch)  # [batch_size, hidden_channels]

        # 3. Apply a final classifier
        x = F.dropout(x, p=0.2, training=self.training)
        x = self.lin(x)
        
        return x

model = GCN(hidden_channels=16)
print(model)

GCN(
  (conv1): GCNConv(2, 16)
  (conv2): GraphConv(16, 16)
  (conv3): GraphConv(16, 16)
  (conv4): GraphConv(16, 16)
  (conv5): GraphConv(16, 16)
  (lin): Linear(in_features=16, out_features=1, bias=True)
)


In [37]:
model = GCN(hidden_channels=256)


In [38]:
model.load_state_dict(torch.load('best_model_0point11_logpred.pt'))

<All keys matched successfully>

In [39]:

print(model)

GCN(
  (conv1): GCNConv(2, 256)
  (conv2): GraphConv(256, 256)
  (conv3): GraphConv(256, 256)
  (conv4): GraphConv(256, 256)
  (conv5): GraphConv(256, 256)
  (lin): Linear(in_features=256, out_features=1, bias=True)
)


In [40]:
# from IPython.display import Javascript
# display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 300})'''))

optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
criterion = torch.nn.MSELoss()###torch.nn.CrossEntropyLoss()
best_loss = float('inf')
def train():
    model.train()

    for data in train_loader:  # Iterate in batches over the training dataset.
        out = model(data.x, data.edge_index, data.batch).float().squeeze()
        #  print(type(out[0]))  # Perform a single forward pass.
        #  print(data.y.shape)
        loss = criterion(out.to(torch.float), data.y.float())  # Compute the loss.
        #  print(loss)
        loss.backward()  # Derive gradients.
        optimizer.step()  # Update parameters based on gradients.
        optimizer.zero_grad()  # Clear gradients.


def test(loader):
    model.eval()

    total_loss = 0
    total_samples = 0
    # criterion = nn.MSELoss()

    with torch.no_grad():
        for data in loader:
            out = model(data.x, data.edge_index, data.batch).float().squeeze()
            loss = criterion(out, data.y)
            total_loss += loss.item() * data.num_graphs
            total_samples += data.num_graphs

    mse = total_loss / total_samples
    return mse
  
for epoch in range(1, 200):
    train()
    train_acc = test(train_loader)
    test_acc = test(test_loader)
    print(f'Epoch: {epoch:03d}, Train MSE: {train_acc:.4f}, Test MSE: {test_acc:.4f}')
    if test_acc < best_loss:
        best_loss = test_acc
        torch.save(model.state_dict(), 'best_model_1week_Nhops.pt')
    

Epoch: 001, Train MSE: 0.1410, Test MSE: 0.1251
Epoch: 002, Train MSE: 0.1383, Test MSE: 0.1169
Epoch: 003, Train MSE: 0.1375, Test MSE: 0.1165
Epoch: 004, Train MSE: 0.1376, Test MSE: 0.1215
Epoch: 005, Train MSE: 0.1367, Test MSE: 0.1157
Epoch: 006, Train MSE: 0.1360, Test MSE: 0.1187
Epoch: 007, Train MSE: 0.1360, Test MSE: 0.1151
Epoch: 008, Train MSE: 0.1356, Test MSE: 0.1149
Epoch: 009, Train MSE: 0.1345, Test MSE: 0.1169
Epoch: 010, Train MSE: 0.1353, Test MSE: 0.1139
Epoch: 011, Train MSE: 0.1340, Test MSE: 0.1158
Epoch: 012, Train MSE: 0.1350, Test MSE: 0.1220
Epoch: 013, Train MSE: 0.1340, Test MSE: 0.1141
Epoch: 014, Train MSE: 0.1332, Test MSE: 0.1162
Epoch: 015, Train MSE: 0.1341, Test MSE: 0.1134
Epoch: 016, Train MSE: 0.1334, Test MSE: 0.1189
Epoch: 017, Train MSE: 0.1333, Test MSE: 0.1143
Epoch: 018, Train MSE: 0.1330, Test MSE: 0.1145
Epoch: 019, Train MSE: 0.1330, Test MSE: 0.1144
Epoch: 020, Train MSE: 0.1331, Test MSE: 0.1188
Epoch: 021, Train MSE: 0.1326, Test MSE:

Epoch: 172, Train MSE: 0.1296, Test MSE: 0.1108
Epoch: 173, Train MSE: 0.1279, Test MSE: 0.1186
Epoch: 174, Train MSE: 0.1273, Test MSE: 0.1137
Epoch: 175, Train MSE: 0.1277, Test MSE: 0.1122
Epoch: 176, Train MSE: 0.1272, Test MSE: 0.1156
Epoch: 177, Train MSE: 0.1274, Test MSE: 0.1126
Epoch: 178, Train MSE: 0.1279, Test MSE: 0.1192
Epoch: 179, Train MSE: 0.1297, Test MSE: 0.1107
Epoch: 180, Train MSE: 0.1270, Test MSE: 0.1150
Epoch: 181, Train MSE: 0.1270, Test MSE: 0.1159
Epoch: 182, Train MSE: 0.1282, Test MSE: 0.1113
Epoch: 183, Train MSE: 0.1276, Test MSE: 0.1185
Epoch: 184, Train MSE: 0.1269, Test MSE: 0.1134
Epoch: 185, Train MSE: 0.1268, Test MSE: 0.1141
Epoch: 186, Train MSE: 0.1270, Test MSE: 0.1131
Epoch: 187, Train MSE: 0.1276, Test MSE: 0.1193
Epoch: 188, Train MSE: 0.1269, Test MSE: 0.1129
Epoch: 189, Train MSE: 0.1267, Test MSE: 0.1146
Epoch: 190, Train MSE: 0.1281, Test MSE: 0.1208
Epoch: 191, Train MSE: 0.1266, Test MSE: 0.1145
Epoch: 192, Train MSE: 0.1270, Test MSE:

In [170]:
gc.collect()

7264